In [ ]:
'''Project utility functions and variables'''

In [ ]:
# import numpy as np
# import astropy.units as u
# import astropy.coordinates as coord

In [ ]:
mystyledict={
    'axes.linewidth': 1.,
    'axes.titlesize': 18.0,
    'axes.labelsize':14,
    
    'figure.dpi': 120.0,
    'figure.figsize': [5,4],
    
    'font.family': 'serif',
    'font.serif':'Liberation Serif',
    
#     'font.serif':'CMU Serif',
#     'axes.unicode_minus':False,
    
    'font.size': 12.0,
    
    'grid.color': 'tab:gray',
    'grid.linewidth': .7,
    
    'legend.fancybox':False,
    'legend.edgecolor':'0.1',
    'lines.linewidth': 1.0,
    'axes.formatter.use_mathtext':True,

    'xtick.bottom': True,
    'xtick.direction': 'in',
    'xtick.major.pad': 7.0,
    'xtick.major.size': 8.0,
    'xtick.major.width': 1.0,
    'xtick.minor.size': 0.0,
    'xtick.minor.width': 1.5,
    'xtick.top': False,
    'ytick.direction': 'in',
    'ytick.left': True,
    'ytick.major.pad': 7.0,
    'ytick.major.size': 8.0,
    'ytick.major.width': 1.0,
    'ytick.minor.size': 0.0,
    'ytick.minor.width': 1.5,
    'ytick.right': False,

    'mathtext.fontset':'cm'
}

In [ ]:
def confidence_ellipse(cov, ax, n_std=3.0, center=(0,0), facecolor="none", **kwargs):
    """
    Create a plot of the covariance confidence ellipse of *x* and *y*.

    Parameters
    ----------
    x, y : array-like, shape (n, )
        Input data.

    ax : matplotlib.axes.Axes
        The axes object to draw the ellipse into.

    n_std : float
        The number of standard deviations to determine the ellipse's radiuses.

    Returns
    -------
    matplotlib.patches.Ellipse

    Other parameters
    ----------------
    kwargs : `~matplotlib.patches.Patch` properties
    """
    from matplotlib.patches import Ellipse
    import matplotlib.transforms as transforms

    pearson = cov[0, 1] / np.sqrt(cov[0, 0] * cov[1, 1])
    # Using a special case to obtain the eigenvalues of this
    # two-dimensionl dataset.
    ell_radius_x = np.sqrt(1 + pearson)
    ell_radius_y = np.sqrt(1 - pearson)
    ellipse = Ellipse(
        (0, 0),
        width=ell_radius_x * 2,
        height=ell_radius_y * 2,
        facecolor=facecolor,
        **kwargs
    )

    # Calculating the stdandard deviation of x from
    # the squareroot of the variance and multiplying
    # with the given number of standard deviations.
    scale_x = np.sqrt(cov[0, 0]) * n_std
    mean_x = center[0]

    # calculating the stdandard deviation of y ...
    scale_y = np.sqrt(cov[1, 1]) * n_std
    mean_y = center[1]

    transf = (
        transforms.Affine2D()
        .rotate_deg(45)
        .scale(scale_x, scale_y)
        .translate(mean_x, mean_y)
    )

    ellipse.set_transform(transf + ax.transData)
    return ax.add_patch(ellipse)


# sigma = azfit.posterior["Sigma_gal"].stack(i=["chain", "draw"]).values
# irand = np.random.randint(0, 4000, 500)
# fig, ax = plt.subplots(
#     1, 2, figsize=(8, 4), subplot_kw={"aspect": "equal"}, sharex=True, sharey=True
# )
# for i in irand:
#     sel = np.meshgrid([0, 1], [0, 1], indexing="ij")
#     confidence_ellipse(sigma[sel[0], sel[1], i], ax[0], edgecolor="gray", alpha=0.05)
#     sel = np.meshgrid([0, 2], [0, 2], indexing="ij")
#     confidence_ellipse(sigma[sel[0], sel[1], i], ax[1], edgecolor="gray", alpha=0.05)
# ax[0].axis([-3,3,-3,3])

In [ ]:
def decompose_T(T):
    """Decompose velocity gradient tensor to interpretable components

    T : array, (3, 3) or (N, 3, 3)
        dv_j / dv_i
    
    """
    if T.ndim == 2:
        T = T[None]
    if T.shape[1:] != (3, 3):
        raise ValueError("`T` must have shape (3, 3) or (N, 3, 3)")
    omegax = 0.5 * (T[:, 2, 1] - T[:, 1, 2])
    omegay = 0.5 * (T[:, 0, 2] - T[:, 2, 0])
    omegaz = 0.5 * (T[:, 1, 0] - T[:, 0, 1])

    w1 = 0.5 * (T[:, 2, 1] + T[:, 1, 2])
    w2 = 0.5 * (T[:, 0, 2] + T[:, 2, 0])
    w3 = 0.5 * (T[:, 1, 0] + T[:, 0, 1])
    w4 = T[:, 0, 0]
    w5 = T[:, 1, 1]
    kappa = (w4 + w5 + T[:, 2, 2]) / 3.0
    T = T.squeeze()
    return dict(
        omegax=omegax,
        omegay=omegay,
        omegaz=omegaz,
        w1=w1,
        w2=w2,
        w3=w3,
        w4=w4,
        w5=w5,
        kappa=kappa,
    )

In [ ]:
def rotate_T_to_galactic(T):
    """R T R^T"""
    # hack rotation matrix from icrs -> galactic
    rotmat = coord.ICRS(
        [1, 0, 0], [0, 1, 0], [0, 0, 1], representation_type=coord.CartesianRepresentation
    ).transform_to(coord.Galactic).cartesian.xyz.value
    rotated_T = np.einsum('ij,njk,kl->nil', rotmat, T, rotmat.T)
    return rotated_T

In [ ]:
def plot_T_icrs(fit, fig=None):
    """Plot 3x3 grid of each component of T = dv/dx
    
    fit : StanFit
        fit object
    """
    if fig:
        ax = fig.axes
    else:
        fig, ax = plt.subplots(3, 3, figsize=(6, 5), sharex=True, sharey=True)
        fig.subplots_adjust(
            bottom=0.15, top=0.92, right=0.95, left=0.1, hspace=0.05, wspace=0.05
        )
        ax = ax.ravel()
    for cax, cT in zip(ax, fit["T_param"].reshape((-1, 9)).T):
        #     cax.hist(cT, bins=32, density=True, histtype="step")
        sns.distplot(cT, hist=False, ax=cax, kde_kws={'lw':1})
        cax.axvline(0, c="gray", lw=0.5)
    fig.text(0.55, 0.05, "m/s/pc", ha="center", va="center", size=20)
    fig.text(0.05, 0.55, "Density", ha="center", va="center", rotation=90, size=20)
    for cax in ax:
        cax.yaxis.set_major_formatter(ticker.NullFormatter())
    for cax in ax:
        cax.set_xticks([-50, 0, 50])
    fig.suptitle("$T$ (ICRS)", size=20)
    return fig

In [ ]:
def plot_omegas(fit):
    """Plot rotational component of T
    """
    wT = decompose_T(fit["T_param"])
    fig, ax = plt.subplots(figsize=(4,4))
    sns.distplot(wT["omegax"], hist=False, kde_kws={'lw':1}, label=r'$\omega_x$')
    sns.distplot(wT["omegay"], hist=False, kde_kws={'lw':1}, label=r'$\omega_y$')
    sns.distplot(wT["omegaz"], hist=False, kde_kws={'lw':1}, label=r'$\omega_z$')
    omega = np.sqrt(wT["omegax"] ** 2 + wT["omegay"] ** 2 + wT["omegaz"] ** 2)
    print(f"omega = {np.mean(omega)} +- {np.std(omega)}")
    sns.distplot(omega, hist=False, color='k', label=r'$\omega$')
    ax.axvline(0, c='k', lw=1)
    ax.legend(fontsize=14)
    ax.set_xlabel(r'$\rm m\,\rm s^{-1}\,\rm pc^{-1}$');
    return fig

In [ ]:
def plot_omegas_galactic(fit):
    """Plot rotational component of T
    """
    wT = decompose_T(rotate_T_to_galactic(fit["T_param"]))
    fig, ax = plt.subplots(figsize=(4,4))
    sns.distplot(wT["omegax"], hist=False, kde_kws={'lw':1}, label=r'$\omega_x$')
    sns.distplot(wT["omegay"], hist=False, kde_kws={'lw':1}, label=r'$\omega_y$')
    sns.distplot(wT["omegaz"], hist=False, kde_kws={'lw':1}, label=r'$\omega_z$')
    omega = np.sqrt(wT["omegax"] ** 2 + wT["omegay"] ** 2 + wT["omegaz"] ** 2)
    print(f"omega = {np.mean(omega)} +- {np.std(omega)}")
    sns.distplot(omega, hist=False, color='k', label=r'$\omega$')
    ax.axvline(0, c='k', lw=1)
    ax.legend(fontsize=14)
    ax.set_xlabel(r'$\rm m\,\rm s^{-1}\,\rm pc^{-1}$');
    return fig

In [ ]:
bovy_oort = dict(A=15.3, B=-11.9, C=-3.2, K=-3.3,
                 stdA=0.4, stdB=0.4, stdC=0.4, stdK=0.6)

In [ ]:
def plot_T_galactic(fit, fig=None, color=None):
    """Plot 3x3 grid of each component of T = dv/dx
    
    fit : StanFit
        fit object
    """
    if fig:
        ax = fig.axes
    else:
        fig, ax = plt.subplots(3, 3, figsize=(6, 5), sharex=True, sharey=True)
        fig.subplots_adjust(
            bottom=0.15, top=0.92, right=0.95, left=0.1, hspace=0.05, wspace=0.05
        )
        ax = ax.ravel()
    for cax, cT in zip(ax, rotate_T_to_galactic(fit["T_param"]).reshape((-1, 9)).T):
        #     cax.hist(cT, bins=32, density=True, histtype="step")
        sns.distplot(cT, hist=False, ax=cax, kde_kws={'lw':1}, color=color)
        cax.axvline(0, c="gray", lw=0.5)
    fig.text(0.55, 0.05, "m/s/pc", ha="center", va="center", size=20)
    fig.text(0.05, 0.55, "Density", ha="center", va="center", rotation=90, size=20)
    for cax in ax:
        cax.yaxis.set_major_formatter(ticker.NullFormatter())
    for cax in ax:
        cax.set_xticks([-50, 0, 50])
    fig.suptitle("$T$ (galactic)", size=20)
    return fig

In [ ]:
def add_transformed_posterior(azfit):
    '''Add transformed posterior samples for convenience
    
    Added parameters:
        - Sigma: velocity dispersion matrix, (3,3)
        - omegax, omegay, omegaz, w1, w1, w2, w3, w5, kappa: decomposed linear velocity field parameters
        - *_gal: quantities rotated to the Galactic frame
    '''
    v = azfit

    for ck, cv in kn.decompose_T(v.posterior['T_param']).items():
        v.posterior[ck]=cv
    # Combine scale and correlation matrix of Sigma to variance matrix
    sigv_samples, Omega_samples = v.posterior['sigv'],  v.posterior['Omega']
    Sigma_samples = np.einsum('cni,cnij,cnj->cnij', sigv_samples, Omega_samples, sigv_samples)
    v.posterior['Sigma'] = ('chain','draw','Simga_dim_0','Sigma_dim_1'), Sigma_samples
    v.posterior['Sigma_gal'] = ('chain','draw','Simga_dim_0','Sigma_dim_1'), kn.rotate_T_to_galactic(Sigma_samples)
    # Add rotated T matrix and decomposition
    v.posterior['T_param_gal'] = ('chain','draw','dim0','dim1'), kn.rotate_T_to_galactic(v.posterior['T_param'])
    for ck, cv in kn.decompose_T(v.posterior['T_param_gal']).items():
        v.posterior[ck+'_gal']  = cv
    return v